### This notebook is designed to generate a set of code-vector targets for a trained AutoEncoder model
- This notebook is step 2 of 3 in setting up a model for generating particle decay event images

## Import Libraries

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

### Import Files from full path on Mayer Machine

In [ ]:
import sys
sys.path.insert(1, '/home/plutku01/projects/particle_generator/')

# Mayer Machine
import ae
import conv_ae
import utils
from dataloader import LArCV_loader

#### Set the GPU to be used for model evaluation
- On Meitner, GPU 1 is the best option

In [ ]:
# Which is best on Mayer?"
device = torch.device(1)

#### Set the root path of the AutoEncoder Experiments Folder

In [ ]:
exp_root = "/home/plutku01/projects/particle_generator/experiments/"

#### Select the model class and append to the experiment root

In [ ]:
model_class = {'mlp': 'mlp_ae/', 'conv':'conv_ae/'}

In [ ]:
model_folder = "larcv_ae/" + model_class['conv']

In [ ]:
exp_root += model_folder

### Find all the experiments in the exp_root folder

In [ ]:
exp_paths = []
for path in os.listdir(exp_root):
    exp_paths.append(os.path.join(exp_root, path))

In [ ]:
print("-"*60)
for i in range(len(exp_paths)):
    exp_name = exp_paths[i].split('/')[-1]
    print("\n{}:".format(str(i)), exp_name, '\n')
    print("-"*60)

### Choose the experiment for evaluation

In [ ]:
exp_dir = exp_paths[0]

In [ ]:
# Create the full path to the experiment
exp_path = os.path.join(exp_root, exp_dir) + "/"
print("Experiment path set as: \n{}".format(exp_path))

In [ ]:
# Set path from where to load the model weights
weights_dir = "weights/"

### Load the model configuration information from the config.csv

In [ ]:
# Load the config csv as a dict
config_csv = exp_path + "config.csv"
config_df = pd.read_csv(config_csv, delimiter = ",")

In [ ]:
# Get the model architecture from config df
num_epochs = int(config_df[config_df['Unnamed: 0'].str.contains("epochs")==True]['0'].values.item())
n_layers = int(config_df[config_df['Unnamed: 0'].str.contains("n_layers")==True]['0'].values.item())
l_dim    = int(config_df[config_df['Unnamed: 0'].str.contains("l_dim")==True]['0'].values.item())
depth    = int(config_df[config_df['Unnamed: 0'].str.contains("depth")==True]['0'].values.item())
im_size  = int(config_df[config_df['Unnamed: 0'].str.contains("dataset")==True]['0'].values.item())**2
im_dim   = int(np.sqrt(im_size))

## Setup PyTorch DataLoader

### Set the paths to the test data and reference training set

#### Test Data

In [ ]:
test_data = "/home/plutku01/data/LArCV/test/larcv_png_{}/".format(im_dim)
num_test_ex = sum( [len(examples) for _, _, examples in os.walk(test_data)] )
print("{} test data will be loaded from: \n{}".format(num_test_ex, test_data))

#### Training Data

In [ ]:
train_data = "/home/plutku01/data/LArCV/train/train_reference/larcv_png_{}/".format(im_dim)
num_train_ex = sum( [len(examples) for _, _, examples in os.walk(train_data)] )
print("{} training data will be loaded from: \n{}".format(num_train_ex, train_data))

#### Setup two instances of a dataloader object

In [ ]:
loader_kwargs = {'num_workers' : 2, 'batch_size': 1, 'shuffle': True}

In [ ]:
transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5],[0.5])])

In [ ]:
test_dataset = LArCV_loader(root = test_data,  transforms = transforms)

In [ ]:
test_loader = DataLoader(test_dataset, **loader_kwargs)

In [ ]:
train_dataset   = LArCV_loader(root = train_data, transforms = transforms)

In [ ]:
train_loader = DataLoader(train_dataset, **loader_kwargs)

### Compute the layer dimensions for the AutoEncoder
- TODO: Need to add function that deals with an AutoEncoder Model trained on MNIST

In [ ]:
# Set up AE layer sizes
if 'mlp' in exp_path:    
    base = [256] 

    # Compute encoder sizes
    sizes = lambda: [ (yield 2**i) for i in range(n_layers) ]
    enc_sizes = base * n_layers
    enc_sizes = [a*b for a,b in zip(enc_sizes, [*sizes()])][::-1]

    # Update kwarg dicts
    # Decoder is the reverse of the encoder
    ae_kwargs = {'enc_sizes' : enc_sizes, 'l_dim' : l_dim, 'im_size' : im_size, 'dec_sizes' : enc_sizes[::-1]}
else:
    # Compute the depth of the feature maps, based on the number of
    # specified layers. If depth is not divisibe by 4, warn
    depth   = [depth] * n_layers
    divisor = lambda: [ (yield 2**i) for i in range(n_layers) ]
    depth   = [a//b for a,b in zip(depth, [*divisor()])][::-1]
        
    # Update kwarg dicts
    # Decoder is the reverse of the encoder
    ae_kwargs = {'enc_depth':[1] + depth, 'dec_depth':depth[1:len(depth)][::-1] + [1],'l_dim':l_dim}

In [ ]:
print("Model l_dim: {}".format(l_dim))
print("Encoder depth: {}".format(ae_kwargs['enc_depth']))
print("Decoder depth: {}".format(ae_kwargs['dec_depth']))

## Load Model from Checkpoint

### Get the list of checkpoint files

In [ ]:
checkpoint_path  = exp_path + weights_dir
checkpoint_names = []
for file in os.listdir(checkpoint_path):
    checkpoint_names.append(os.path.join(checkpoint_path, file))

In [ ]:
# We wish to save a list of labels for ease of plot labelling later
checkpoint_name_labels = []

In [ ]:
print("-"*60)
for i in range(len(checkpoint_names)):
    name = checkpoint_names[i].split('/')[-1]
    checkpoint_name_labels.append(name)
    print("\n{}:".format(str(i)), name, '\n')
    print("-"*60)

### Select the checkpoint from which you wish to generate code vector targets

In [ ]:
index = 2
current_checkpoint = checkpoint_names[index]
current_checkpoint_label = checkpoint_name_labels[index]

### Load the checkpoint file using PyTorch

In [ ]:
# Load the model checkpoint
# Keys: ['state_dict', 'epoch', 'optimizer']
checkpoint = torch.load(current_checkpoint)

### Set up the model on the GPU

In [ ]:
# Set up model on GPU
if 'mlp' in exp_path:
    model = ae.AutoEncoder(**ae_kwargs).to(device)
else:
    model = conv_ae.ConvAutoEncoder(**ae_kwargs).to(device)

### Load the model's state dictionary

In [ ]:
model.load_state_dict(checkpoint['state_dict'], strict=False)

### Put model in evaluation mode

In [ ]:
model.eval()

## Generate a set of code vector targets

### If evaluating convolutional model, get the size of the code tensor using a random torch tensor

In [ ]:
if 'conv' in exp_path:
    code = l_dim
    '''
    x = torch.randn(1, 1, im_dim, im_dim).to(device)
    x = model.encoder(x)
    code_dim = (x.shape[1], x.shape[2], x.shape[3])
    '''
    print("Code vector dimension is: {}".format(code)) #"[{} x {} x {}] = {}".format(code_dim[0], 
                                                        #       code_dim[1], 
                                                        #       code_dim[2], 
                                                        #       code_dim[0]*code_dim[1]*code_dim[2]))

#### Create directory for saving code layer output targets

In [ ]:
# Specify the directory where the code vectors should be saved
deploy_dir = "/home/plutku01/projects/particle_generator/code_vectors_target/"

In [ ]:
if 'mlp' in exp_path:
    deploy_dir += "mlp_ae/code_vectors_{}_{}/".format(im_dim, l_dim)
else:
    deploy_dir += "conv_ae/code_vectors_{}_{}/".format(im_dim, code) #code_dim[0]*code_dim[1]*code_dim[2])

In [ ]:
# Create the save directory, if it doesn't already exist
os.mkdir(deploy_dir)

In [ ]:
if 'mlp' in exp_path:
    deploy_dir += "code_vectors_{}_{}/".format(im_dim, l_dim)
else:
    deploy_dir += "code_vectors_{}_{}/".format(im_dim, code) #code_dim[0]*code_dim[1]*code_dim[2])

In [ ]:
os.mkdir(deploy_dir)

In [ ]:
print("Code vector targets will be saved to:\n{}".format(deploy_dir))

### Generate deploy targets by looping over the dataloader using only the encoder

In [ ]:
# Codes will be a list of numpy arrays of 32-bit floats
codes = []

In [ ]:
for idx, image in enumerate(train_loader):

    # Flatten image into a vector, if mlp
    if 'mlp' in exp_path:
        image = image.view(loader_kwargs['batch_size'], -1).to(device)
    else:
        image = image.to(device)
    
    # Get the output of just the encoder
    code = model.encoder(image)
    
    # If using conv model -- flatten tensor
    if 'conv' in exp_path:
        code = code.view(loader_kwargs['batch_size'], -1).to(device)

    # Save the output tensor to a list
    codes.append(code.detach().cpu().numpy())

In [ ]:
print("Generated {} code vectors from {} training images".format(len(codes), len(train_loader)))

In [ ]:
codes[0].size

## Save the list of code vectors to disk
- We then use the built-in np.save function to store the np array as a .npy file
- The numpy array of floats can be read back losslessly using np.load("float_file.npy")
- This process will require the downstream creation of a [custom dataset](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html)

In [ ]:
for idx, arr in enumerate(codes):
    file_name = deploy_dir + "target_{}.npy".format(idx)
    np.save(file_name, arr)

##### Check that saving actually occurred

In [ ]:
num_vecs = sum( [len(examples) for _, _, examples in os.walk(deploy_dir)] )

In [ ]:
if num_vecs == len(codes):
    print("Saved {} code vectors to \n{}".format(num_vecs, deploy_dir))
else:
    print("Something went wrong...")

## Generate a set of test vectors using the test set
- These vectors will be used to determine the stopping criterion for the ewm generator model

#### Create directory for saving code layer output test targets

In [ ]:
# Specify the directory where the code vector test targets should be saved
deploy_dir = "/home/plutku01/projects/particle_generator/code_vectors_test/"

In [ ]:
if 'mlp' in exp_path:
    deploy_dir += "mlp_ae/code_vectors_{}_{}/".format(im_dim, l_dim)
else:
    deploy_dir += "conv_ae/code_vectors_{}_{}/".format(im_dim, code) # code_dim[0]*code_dim[1]*code_dim[2])

In [ ]:
# Create the save directory, if it doesn't already exist
os.mkdir(deploy_dir)

In [ ]:
if 'mlp' in exp_path:
    deploy_dir += "code_vectors_{}_{}/".format(im_dim, l_dim)
else:
    deploy_dir += "code_vectors_{}_{}/".format(im_dim, code) # code_dim[0]*code_dim[1]*code_dim[2])

In [ ]:
os.mkdir(deploy_dir)

In [ ]:
print("Test vectors will be saved to:\n{}".format(deploy_dir))

In [ ]:
# Codes will be a list of numpy arrays of 32-bit floats
codes = []

In [ ]:
for idx, image in enumerate(test_loader):

    # Flatten image into a vector, if mlp
    if 'mlp' in exp_path:
        image = image.view(loader_kwargs['batch_size'], -1).to(device)
    else:
        image = image.to(device)
    
    # Get the output of just the encoder
    code = model.encoder(image)
    
    # If using conv model -- flatten tensor
    if 'conv' in exp_path:
        code = code.view(loader_kwargs['batch_size'], -1).to(device)

    # Save the output tensor to a list
    codes.append(code.detach().cpu().numpy())

In [ ]:
print("Generated {} code vector test targets from {} test images".format(len(codes), len(test_loader)))

In [ ]:
codes[0].size

## Save the list of code vector test targets to disk

In [ ]:
for idx, arr in enumerate(codes):
    file_name = deploy_dir + "target_{}.npy".format(idx)
    np.save(file_name, arr)

In [ ]:
num_vecs = sum( [len(examples) for _, _, examples in os.walk(deploy_dir)] )

In [ ]:
if num_vecs == len(codes):
    print("Saved {} code vectors to \n{}".format(num_vecs, deploy_dir))
else:
    print("Something went wrong...")